# Fetching Tweets in Real Time using Twitter API

Tweepy library is used to route all the Live tweets to a TCP socket server.<br> 
This server is hosted in same local machine.

**Import tweepy library**

In [1]:
! pip3 install tweepy --user

In [2]:
import tweepy

**Import OAuthHandler**<br> Used for Authentication protocol (OAuth) of Twitter API.

In [3]:
from tweepy import OAuthHandler

**Import Stream & StreamListner**<br> Listen to streaming data getting from the socket.

In [4]:
from tweepy import Stream
from tweepy.streaming import StreamListener

**Import socket** <br> It is used to create socket connection with Localhost i.e. local server. <br>
**Import Json** <br>Json is used because twitter data comes in json format.

In [5]:
import socket
import json

To connect to Twitter API, we need these 4 Keys. <br>
It can be generated from https://apps.twitter.com/ by creating a new app.

In [6]:
consumer_key = 
consumer_secret = 
access_token = 
access_secret = 

#### Class TweetsListener
    1.listening to the stream via StreamListener. 
    2.constrctor function (init) : initializing the socket. 
    3.tweet_data contains the tweet's json coming from the stream. 
    4.In the tweet's json object 'text' field contains the actual Tweet.
    5.The actual tweet is extracted & sent to the client socket.
    6.on_error is used to catch & throw errors.

In [7]:
class TweetsListener(StreamListener):
  def __init__(self, csocket):
      self.client_socket = csocket
  def on_data(self, tweet_json):
      try:
          tweet_data = json.loads( tweet_json )
          print("tweet_text: ", tweet_data['text'].encode('utf-8') )
          print("created_at: ", tweet_data['created_at'])  
          print("name: ", tweet_data['user']['name'])              
          print("location: ", tweet_data['user']['location'])
          print("\n")
          s = tweet_data['text'] + "\n"
          print(s)
          s = s.encode('utf-8')
          self.client_socket.send( s )
          return True
      except BaseException as e:
          print("Error on_data: %s" % str(e))
      return True
  def on_error(self, status):
      print(status)
      return True

### Function send_twitter_data():
    1. auth is doing Authentication check using OAuthHandler. It uses 4 keys of Twitter APIs.
    2. consumer_key & consumer_secret are like username & access_token & access_secret are like password.
    3. twitter_stream will get the actual twitter live stream data.
    4. It'll call Stream with auth & call the Class TweetsListner & pass c_socket info to the class.
    5. From the stream of tweets, filter & take only tweets which contains desired word (Here - vaccine).

In [8]:
def send_twitter_data(c_socket):
  auth = OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_secret)
  twitter_stream_data = Stream(auth, TweetsListener(c_socket))
  twitter_stream_data.filter(track=['vaccine'])

### Socket Creation:
    1. Socket is created by using socket function.
    2. Host is localhost, which is machine IP, where this Jupyter notebook is running.
    3. Port is 7773 (It can be anything, unique for this service)
    4. Now bind host & port address to the socket.

In [9]:
# Creating a socket object
s = socket.socket()
# Geting local machine name
host = "127.0.0.1"
# Reserving a port for your service.
port = 7777
# Binding to the port
s.bind((host,port))
print("Listening on port: %s" % str(port))

Listening on port: 7777


socket will wait & listen for few seconds. <br>
After that connection is made.

In [10]:
# Waiting for client connection.
s.listen(5)

In [11]:
# Establishing connection with client.
c, addr = s.accept()

In [12]:
print("Received Request from: " + str(addr))

Received Request from: ('127.0.0.1', 37948)


### Send live Tweets in real time to the Socket
Function send_twitter_data will send all the Tweets containing desired word to socket 'c' in real time.<br>
This will be consumed in the "HashTag_Analysis" jupyter notebook of Spark Streaming to do Trend Analysis.

In [13]:
send_twitter_data(c)

tweet_text:  b'RT @PAHealthDept: #COVID19 Update (as of 2/12/21 at 12:00 am):\n\xe2\x80\xa2 4,088 additional positive cases of COVID-19\n\xe2\x80\xa2 892,344 total cases statewid\xe2\x80\xa6'
created_at:  Sat Feb 13 17:09:39 +0000 2021
name:  Kayla Huppman
location:  PA


RT @PAHealthDept: #COVID19 Update (as of 2/12/21 at 12:00 am):
• 4,088 additional positive cases of COVID-19
• 892,344 total cases statewid…

tweet_text:  b'\xf0\x9f\x98\x8a\xf0\x9f\x98\x8a\xf0\x9f\x98\x8a\xf0\x9f\x98\x8a\xf0\x9f\x98\x8a'
created_at:  Sat Feb 13 17:09:40 +0000 2021
name:  Joseph (il/lui) ⚪🔴⚪ 🦆
location:  None


😊😊😊😊😊

tweet_text:  b'RT @GillianHTurner: Some exciting personal news today\n1. My dad got his 1st covid vaccine\n2. My husband officially became a US citizen \xf0\x9f\x87\xba\xf0\x9f\x87\xb8(c\xe2\x80\xa6'
created_at:  Sat Feb 13 17:09:40 +0000 2021
name:  Don 🚬🍷
location:  Los Angeles - California 


RT @GillianHTurner: Some exciting personal news today
1. My dad got his 1st covid vaccine
2. 

KeyboardInterrupt: 